In [40]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [41]:
!wget -P /tmp/ https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
!ls /tmp/

--2025-08-06 20:14:43--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: '/tmp/spacex_launch_dash.csv.2'

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2025-08-06 20:14:44 (590 MB/s) - '/tmp/spacex_launch_dash.csv.2' saved [2476/2476]

68796f648840d
687b0bbf909fa
Sublime Text.4cff18d2bab96a93366319a9e0fa060d.94967930a6d7efa1ec47f82db33908d6.sock
Tor ProjectUpdateLock-191289357BF03661
__KMP_REGISTERED_LIB_44598
com.apple.launchd.RnGOcbMFq5
com.sogou.inputmethod
spacex_launch_das

In [42]:
# Read the airline data into pandas dataframe
spacex_df=pd.read_csv('/tmp/spacex_launch_dash.csv')
#spacex_df.head()
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [43]:
# Create a dash application
app = dash.Dash(__name__)


In [44]:
# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
                style={'textAlign': 'center', 'color': '#503D36','font-size': 40}),
                # TASK 1: Add a dropdown list to enable Launch Site selection
                # The default select value is for ALL sites
                html.Div([
                    html.Label("Select Launch Site:"),
                    dcc.Dropdown(
                        id='site-dropdown', 
                        options=[
                            {'label': 'All Sites', 'value': 'ALL'},
                            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                        ],
                        value='ALL',
                        placeholder="Select a Launch Site here",
                        searchable=True
                        #style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
                        )
                    ], style={'width': '80%', 'padding': '10px', 'margin': 'auto'}),
                        

                    # TASK 2: Add a pie chart to show the total successful launches count for all sites
                    # If a specific launch site was selected, show the Success vs. Failed counts for the site
                    html.Div(dcc.Graph(id='success-pie-chart'), style={'width': '80%', 'margin': 'auto'}),
                    html.Br(),

                    
                    # TASK 3: Add a slider to select payload range
                    html.Div([
                        html.Label("Select Payload Mass Range (Kg):"),
                        dcc.RangeSlider(
                            id='payload-slider',
                            min=0,
                            max=10000,
                            step=500,
                            value=[min_payload, max_payload],
                            marks={
                                0: '0',
                                1000: '1000',
                                2000: '2000',
                                3000: '3000',
                                4000: '4000',
                                5000: '5000',
                                6000: '6000',
                                7000: '7000',
                                8000: '8000',
                                9000: '9000',
                                10000: '10000'
                            }
                            #marks={i: f'{i}' for i in range(spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max())},
                            #value=[min_payload, max_payload]
                        )
                    ], style={'width': '80%', 'margin': 'auto', 'padding': '20px'}),

                    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                    html.Div(dcc.Graph(id='success-payload-scatter-chart'), style={'width': '80%', 'margin': 'auto'}),
])


In [45]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # Show total successful launches for all sites
        fig = px.pie(
            spacex_df[spacex_df['class'] == 1],
            names='Launch Site',
            title='Total Successful Launches by Site'
        )
        return fig
    else:
        # Filter dataframe for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]

        # Count success vs. failure for the site
        site_counts = filtered_df['class'].value_counts().reset_index()
        site_counts.columns = ['class', 'count']
        site_counts['class'] = site_counts['class'].map({1: 'Success', 0: 'Failure'})

        # Show success vs. failure counts for the selected site
        fig = px.pie(
            site_counts,
            values='count',
            names='class',
            title=f'Success vs. Failure Counts for {entered_site}',
            color='class',
            color_discrete_map={'Success': 'green', 'Failure': 'red'}
        )
        return fig

In [47]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_scatter_chart(entered_site, payload_range):
    # Filter by payload range first
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]

    # Further filter by launch site if not 'ALL'
    if entered_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]

    # Create the scatter plot
    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Launch Site',
        title='Payload Mass vs. Launch Outcome',
        labels={'Class': 'Launch Outcome (1=Success, 0=Failure)'}
    )
    return fig

In [48]:
# Run the app
if __name__ == '__main__':
    app.run()